# reference
[example](https://github.com/peterdsharpe/AeroSandbox/blob/master/tutorial/06%20-%20Aerodynamics/01%20-%20AeroSandbox%203D%20Aerodynamics%20Tools/01%20-%20Vortex%20Lattice%20Method.ipynb)

[Thesis documentation](https://peterdsharpe.github.io/AeroSandbox/tutorial/sharpe-pds-sm-AeroAstro-2021-thesis.pdf)

[developer documentation]

In [2]:
import aerosandbox as asb
import aerosandbox.numpy as np
%matplotlib notebook

# Vad vill vi egentligen göra?
1. Hitta en flygplansprofil som ger ca 800g lyftkraft i 15-20km/h
1. minimera turbolens/drag för att få så lång flygtid som möjligt

In [4]:
wing_airfoil = asb.Airfoil("sd7037")

In [199]:
wing_params = {'wingspan_m': 1.0,
               'deg_vshape':0,
               'deg_arrow':20,
               
               'base_cord':0.2,
               'base_airfoil':asb.Airfoil("sd7037"),
               'base_twist': 0,
               
               'wingtip_cord': 0.1,
               'wingtip_airfoil': asb.Airfoil("sd7037"),
               'wingtip_twist': -3,
              
               'winglet_cord':0.05,
               'winglet_length':0.10,
               'winglet_angle':45,
               'winglet_x_offset':0.05,
               'winglet_airfoil': asb.Airfoil("naca0010")}

In [200]:
# x: bakåt
# y: längs med vingen
# z: höjd

def build_wing(params):
    airfoil_base = asb.WingXSec(  # Root
                    xyz_le=[0, 0, 0],  # Coordinates of the XSec's leading edge, relative to the wing's leading edge.
                    chord=params['base_cord'],
                    twist=params['base_twist'],  # degrees
                    airfoil=params['base_airfoil'])

    py = params['wingspan_m']/2
    px = np.sin(params['deg_arrow']/180*np.pi)*py
    pz = np.sin(params['deg_vshape']/180*np.pi)*py

    airfoil_tip = asb.WingXSec(  # Root
                    xyz_le=[px, py, pz],  # Coordinates of the XSec's leading edge, relative to the wing's leading edge.
                    chord=params['wingtip_cord'],
                    twist=params['wingtip_twist'],  # degrees
                    airfoil=params['wingtip_airfoil'])
    
    
    wx =  px + params['wingtip_cord'] - params['winglet_cord'] + params['winglet_x_offset']
    wy = py + np.cos(params['winglet_angle']/180*np.pi) * params['winglet_length']
    wz = np.sin(params['winglet_angle']/180*np.pi) * params['winglet_length'] + pz
    winglet  = asb.WingXSec(xyz_le=[wx, wy, wz],
                            chord = params['winglet_cord'],
                            twist=0,
                        airfoil=params['winglet_airfoil'])
    return [airfoil_base, airfoil_tip, winglet]



In [201]:
main_wing  = build_wing(wing_params)

In [202]:
def plot_wing(wing_params, plot_type='static'):
    """
    wing_params: dict containing all parameters to build a simple wing
    plot_type: 'static' to be able to show plots in github, 'panel' for interactive plot 
    """
    main_wing  = build_wing(wing_params)

    airplane = asb.Airplane(
        name="OdDa-wing",
        xyz_ref=[0, 0, 0],  # CG location
        wings=[asb.Wing(
                name="Main Wing",
                symmetric=True,  # Should this wing be mirrored across the XZ plane?
                xsecs=main_wing,
                )
            ]
    )

    vlm = asb.VortexLatticeMethod(
        airplane=airplane,
        op_point=asb.OperatingPoint(
            velocity=25,  # m/s
            alpha=5,  # degree
        )
    )
    aero = vlm.run()
    vlm.draw(show_kwargs=dict(jupyter_backend='panel'))
    for k, v in aero.items():
        print(f"{k.rjust(4)} : {v}")


In [203]:
plot_wing(wing_params)

   L : 36.05112821369475
   D : 0.7580259604962771
   Y : 4.754938034024631e-16
   l : -6.099734962609532e-16
   m : -5.097825954506293
   n : 3.567422980423862e-17
  CL : 0.5715293040474392
  CD : 0.012017211974180498
  CY : 7.538145295387637e-18
  Cl : -8.071763416534965e-18
  Cm : -0.5441873342269918
  Cn : 4.720761554592607e-19
 F_g : [-2.38692141e+00  4.75493803e-16  3.59800091e+01]
 F_w : (-0.7580259604962771, 4.754938034024631e-16, -36.05112821369475)
 M_g : [ 6.10761577e-16 -5.09782595e+00  1.76242145e-17]
 M_w : (-6.099734962609532e-16, -5.097825954506293, 3.567422980423862e-17)


In [113]:
import ipywidgets as widgets
from IPython.display import display

In [122]:
def widgets_to_dict():
    wing_params = {'wingspan_m': w_wingsgspan.value,
               'deg_vshape':w_deg_vshape.value,
               'deg_arrow':w_deg_arrow.value,
               
               'base_cord':w_base_cord.value,
               'base_airfoil':asb.Airfoil("sd7037"),
               'base_twist': w_base_twist.value,
               
               'wingtip_cord': w_wingtip_cord.value,
               'wingtip_airfoil': asb.Airfoil("sd7037"),
               'wingtip_twist': w_wingtip_twist.value,
              
               'winglet_cord': w_winglet_cord.value,
               'winglet_length':w_winglet_length.value,
               'winglet_angle':w_winglet_angle.value,
               'winglet_airfoil': asb.Airfoil("naca0010")}
    
    return wing_params

w_wingsgspan = widgets.FloatSlider(min=0.5, max=3.0, description='Wing Span m')
w_deg_vshape = widgets.FloatSlider(min=0, max=50, description='deg V-shape')
w_deg_arrow = widgets.FloatSlider(min=0,max=80, description='deg arrow')
w_base_cord = widgets.FloatSlider(min=0.1,max=1.0, description='base chord')
w_base_twist = widgets.FloatSlider(min=-20,max=20, description='base twist')

w_wingtip_cord = widgets.FloatSlider(min=0.1,max=1.0, description='wingtip chord')
w_wingtip_twist = widgets.FloatSlider(min=-20,max=20, description='wingtip twist')

w_winglet_cord = widgets.FloatSlider(min=0.1,max=1.0, description='winglet chord')
w_winglet_length = widgets.FloatSlider(min=0,max=1.0, description='winglet length')
w_winglet_angle = widgets.FloatSlider(min=-90,max=90, description='winglet angle')

button = widgets.Button(description="Plot Airplane")
output = widgets.Output()

def on_button_click(b):
    with output:
        wing_params = widgets_to_dict()
        plot_wing(wing_params)
            
button.on_click(on_button_click)

display(w_wingsgspan, 
        w_deg_vshape, 
        w_deg_arrow,
       w_base_cord,
       w_base_twist,
       w_wingtip_cord,
       w_wingtip_twist,
       w_winglet_cord,
       w_winglet_length,
       w_winglet_angle,
       button)

FloatSlider(value=0.5, description='Wing Span m', max=3.0, min=0.5)

FloatSlider(value=0.0, description='deg V-shape', max=50.0)

FloatSlider(value=0.0, description='deg arrow', max=80.0)

FloatSlider(value=0.1, description='base chord', max=1.0, min=0.1)

FloatSlider(value=0.0, description='base twist', max=20.0, min=-20.0)

...

FloatSlider(value=0.0, description='wingtip twist', max=20.0, min=-20.0)

FloatSlider(value=0.1, description='winglet chord', max=1.0, min=0.1)

FloatSlider(value=0.0, description='winglet length', max=1.0)

FloatSlider(value=0.0, description='winglet angle', max=90.0, min=-90.0)

Button(description='Plot Airplane', style=ButtonStyle())

In [120]:
def widgets_to_dict():
    wing_params = {'wingspan_m': w_wingsgspan.value,
               'deg_vshape':w_deg_vshape.value,
               'deg_arrow':w_deg_arrow.value,
               
               'base_cord':w_base_cord.value,
               'base_airfoil':asb.Airfoil("sd7037"),
               'base_twist': w_base_twist.value,
               
               'wingtip_cord': w_wingtip_cord.value,
               'wingtip_airfoil': asb.Airfoil("sd7037"),
               'wingtip_twist': w_wingtip_twist.value,
              
               'winglet_cord': w_winglet_cord.value,
               'winglet_length':w_winglet_length.value,
               'winglet_angle':w_winglet_angle.value,
               'winglet_airfoil': asb.Airfoil("naca0010")}
    
    return wing_params
    

In [ ]:
wing_params = {'wingspan_m': 1.0,
               'deg_vshape':0,
               'deg_arrow':20,
               
               'base_cord':0.2,
               'base_airfoil':asb.Airfoil("sd7037"),
               'base_twist': 0,
               
               'wingtip_cord': 0.1,
               'wingtip_airfoil': asb.Airfoil("sd7037"),
               'wingtip_twist': -3,
              
               'winglet_cord':0.05,
               'winglet_length':0.10,
               'winglet_angle':70,
               'winglet_airfoil': asb.Airfoil("naca0010")}

In [ ]:
widgets

In [11]:
cord1 = 0.5
l1 = 0.4
x1 = 0.3
cord2 = 0.2
l2 = 0.7
x2 = 0.5
cord3 = 0.15

airplane = asb.Airplane(
    name="OdDa-wing",
    xyz_ref=[0, 0, 0],  # CG location
    wings=[asb.Wing(
            name="Main Wing",
            symmetric=True,  # Should this wing be mirrored across the XZ plane?
            xsecs=[  # The wing's cross ("X") sections
                asb.WingXSec(  # Root
                    xyz_le=[0, 0, 0],  # Coordinates of the XSec's leading edge, relative to the wing's leading edge.
                    chord=cord1,
                    twist=2,  # degrees
                    airfoil=wing_airfoil,  # Airfoils are blended between a given XSec and the next one.
                ),
                asb.WingXSec(  # Mid
                    xyz_le=[x1, l1, 0],
                    chord=cord2,
                    twist=0,
                    airfoil=wing_airfoil,
                ),
                asb.WingXSec(  # Tip
                    xyz_le=[x2, l2, 0.0],
                    chord=cord3,
                    twist=-2,
                    airfoil=wing_airfoil,
                ),
                ],
            )
        ]
)


In [98]:
vlm = asb.VortexLatticeMethod(
    airplane=airplane,
    op_point=asb.OperatingPoint(
        velocity=25,  # m/s
        alpha=5,  # degree
    )
)



In [105]:
asb.OperatingPoint?

In [99]:
aero = vlm.run()  # Returns a dictionary
for k, v in aero.items():
    print(f"{k.rjust(4)} : {v}")


   L : 36.168319774011515
   D : 0.8057240409855972
   Y : -2.7643428453412444e-16
   l : -6.237838284556969e-16
   m : -3.594861138253998
   n : -3.3058906947422264e-17
  CL : 0.5777202516506135
  CD : 0.012869912084044834
  CY : -4.4155129523231034e-18
  Cl : -8.43110050538014e-18
  Cm : -0.38530695510392315
  Cn : -4.468262150395306e-19
 F_g : [-2.34961876e+00 -2.76434285e-16  3.61009119e+01]
 F_w : (-0.8057240409855972, -2.7643428453412444e-16, -36.168319774011515)
 M_g : [ 6.18528869e-16 -3.59486114e+00  8.72994507e-17]
 M_w : (-6.237838284556969e-16, -3.594861138253998, -3.3058906947422264e-17)


In [100]:
vlm.draw(show_kwargs=dict(jupyter_backend='panel'))

In [1]:
from pyvista import demos

# basic glyphs demo
mesh = demos.glyphs(2)

text = demos.logo.text_3d("I'm interactive!", depth=0.2)
text.points *= 0.1
text.translate([0, 1.4, 1.5], inplace=True)
mesh += text
mesh['Example Scalars'] = mesh.points[:, 0]

mesh.plot(cpos='xy', jupyter_backend='ipygany', show_scalar_bar=True)


AppLayout(children=(VBox(children=(HTML(value='<h3>Example Scalars</h3>'), Dropdown(description='Colormap:', o…

In [4]:
from pyvista import demos
demos.plot_logo(jupyter_backend='panel')